<a href="https://colab.research.google.com/github/Keita062/work/blob/main/Eye(11.12night).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [21]:
Eye = pd.read_csv('/content/DO - コピー.csv')

In [22]:
Eye.head(1)

,DataCount,TIME(Y/M/D h:m:s),milliSeconds,leftX,leftY,rightX,rightY,leftPSize,rightPSize
0,2,2024/11/2 18:13,535,770.993,542.5826,744.4664,567.8753,16.6672,17.0328


In [23]:
want = Eye.iloc[:, 2:9]
want.head()

,milliSeconds,leftX,leftY,rightX,rightY,leftPSize,rightPSize
0,535,770.9930,542.5826,744.4664,567.8753,16.6672,17.0328
1,581,783.1512,470.1998,757.0742,569.6154,14.4211,14.9066
2,612,759.7722,475.1764,756.7003,557.0222,14.3276,14.7274
3,644,775.8635,485.2859,749.9686,561.2388,14.6416,14.8207
4,676,778.5135,487.6267,752.9910,566.4250,14.2361,15.1852


In [24]:
# 7列中4列以上が0の行をすべて0にする
want[(want == 0).sum(axis=1) >= 4] = 0

In [25]:
# 7列中4列未満が0の行で0でない前の行まで遡って補完
for i in range(1, len(want)):  # 2行目以降で実行
    if (want.iloc[i] == 0).sum() < 4:  # 0が4列未満の行のみ
        for col in want.columns:  # 各列ごとに補完を確認
            if want.iloc[i][col] == 0:
                # 0でない値を見つけるまで遡って補完
                j = i - 1
                while j >= 0 and want.iloc[j][col] == 0:
                    j -= 1
                if j >= 0:  # 0でない値が見つかった場合
                    want.at[i, col] = want.iloc[j][col]  # 0でない値を設定

In [26]:
# 3行以上連続して0がある部分を「まばたき」として検出
blink_rows = []
zero_count = 0

for i in range(len(want)):
    if (want.iloc[i] == 0).all():  # 行がすべて0の場合
        zero_count += 1
    else:
        if zero_count >= 3:
            blink_rows.append((i - zero_count, i - 1))  # まばたきの行範囲を記録
        zero_count = 0

# 最後の行が連続する0で終わっていた場合も処理
if zero_count >= 3:
    blink_rows.append((len(want) - zero_count, len(want) - 1))

print("Blinks detected at rows:", blink_rows)

Blinks detected at rows: [(143, 150), (235, 240), (273, 279), (632, 635), (741, 744), (945, 949), (1130, 1136), (1317, 1319), (1525, 1531), (1623, 1629), (1643, 1650), (1812, 1816), (1872, 1876), (2057, 2061), (2206, 2209), (2392, 2398)]


In [28]:
want.to_csv('result.csv', index=False)

In [29]:
# 瞬きの数をカウントする変数
blink_count = 0
zero_count = 0  # 連続して0が続いている行数

# 行ごとにすべての列が0かを確認
for i in range(len(want)):
    if (want.iloc[i] == 0).all():  # 行のすべての列が0の場合
        zero_count += 1
    else:
        if zero_count >= 3:  # 3行以上連続して0が続いていた場合
            blink_count += 1
        zero_count = 0  # 連続カウントをリセット

# データフレームの最後まで0が連続していた場合のチェック
if zero_count >= 3:
    blink_count += 1

print("瞬きの数:", blink_count)

瞬きの数: 16
